In [54]:
##complex data와 똑같이 40ms로 parsing했음 정확도는 10ms와 비슷하게 나옴

import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob


#filename queue
filename_queue=glob.glob('./rawdata/data[5].csv')

In [55]:
#Parsing data

training_set=[]
test_set=[]
training_label=[]
test_label=[]
label_count=np.zeros(16)
traininglabel_count=np.zeros(16)
testlabel_count=np.zeros(16)

#return_file=open(return_path,'a')
#writer=csv.writer(return_file)
#writer.writerow()
n_data=[]

for filename in filename_queue:
    file=open(filename, newline='')
    
    reader=csv.reader(file)
    header=next(reader)
    
    for row in reader:
        row[4:]=[float(i) for i in row[4:]]
        date=datetime.strptime(row[0],'%Y/%m/%d')
        msec=row[1:3]
        labl=int(row[3])
        label_count[labl]+=1
        X_acc=list(row[4*i+4] for i in range(150))
        Y_acc=list(row[4*i+604] for i in range(150))
        Z_acc=list(row[4*i+1204] for i in range(150))
        X_gyro=list(row[4*i+1804] for i in range(150))
        Y_gyro=list(row[4*i+2404] for i in range(150))
        Z_gyro=list(row[4*i+3004] for i in range(150))
    
        window=np.array([[X_acc[0:25],X_acc[25:50],X_acc[50:75],X_acc[75:100],X_acc[100:125],X_acc[125:150]],
                [Y_acc[0:25],Y_acc[25:50],Y_acc[50:75],Y_acc[75:100],Y_acc[100:125],Y_acc[125:150]],
                [Z_acc[0:25],Z_acc[25:50],Z_acc[50:75],Z_acc[75:100],Z_acc[100:125],Z_acc[125:150]],
                [X_gyro[0:25],X_gyro[25:50],X_gyro[50:75],X_gyro[75:100],X_gyro[100:125],X_gyro[125:150]],
                [Y_gyro[0:25],Y_gyro[25:50],Y_gyro[50:75],Y_gyro[75:100],Y_gyro[100:125],Y_gyro[125:150]],
                [Z_gyro[0:25],Z_gyro[25:50],Z_gyro[50:75],Z_gyro[75:100],Z_gyro[100:125],Z_gyro[125:150]]])
    
            #5 features  what else?2
        window_mean=window.mean(axis=-1)
        window_stddev=window.std(axis=-1)
        window_median=np.median(window,axis=-1)
        window_percent25=np.percentile(window,25,axis=-1)
        window_percent75=np.percentile(window,75,axis=-1)

        window_feature=np.array([[window_mean],[window_stddev],[window_median],[window_percent25],[window_percent75]])
        window_feature=window_feature.reshape(180)
    
        if (label_count[labl]%5!=1):
            training_set.append(window_feature)
            training_label.append(labl)
            traininglabel_count[labl]+=1
        else:
            test_set.append(window_feature)
            test_label.append(labl)
            testlabel_count[labl]+=1
        

In [56]:
#print(window_feature.shape)
#print(training_label)
print(test_set)

[array([-9.08506960e-01, -1.07447312e+00,  7.36893560e-01, -1.31863664e+00,
       -2.04701188e+00, -5.87390160e-01,  6.88099220e+00, -8.07338440e-01,
        1.26341200e-02,  8.68690280e-01, -5.35418000e-01,  6.90367616e+00,
        4.84211580e+00, -8.76979056e+00, -9.77975328e+00,  5.77149000e-01,
        1.39386708e+00,  5.96032972e+00,  8.12896018e+01,  1.24012000e+01,
        1.39131998e+01, -5.73524000e+01,  4.81264014e+01,  4.94452000e+01,
        2.67316004e+01,  2.86916002e+01,  7.21280024e+00, -1.96011202e+02,
        7.55439852e+00,  4.65052003e+01, -3.36223996e+01, -4.99184002e+01,
        1.44199996e+00,  1.61168002e+01, -4.39544020e+01,  4.52900002e+01,
        2.44576123e+00,  2.11524040e+00,  8.95318237e-01,  3.81432425e+00,
        6.21636340e+00,  1.64460640e+00,  2.30915416e+00,  2.93148785e+00,
        7.71613576e-01,  1.96959123e+00,  2.29968323e+00,  1.96832552e+00,
        6.56404465e+00,  2.19541198e+00,  8.78771347e-01,  9.58763919e+00,
        1.20588649e+01, 

In [59]:
#SVM
#prepare the data
X=training_set
y=training_label
#prepare the model
clf = sklearn.svm.SVC(gamma='scale',tol=0.1)

clf.fit(X,y)
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

test_pred1=clf.predict(test_set)

ohc=OneHotEncoder(categories=[range(16)])
onehot_pred1=test_pred1.reshape(-1,1)
onehot_pred1=ohc.fit_transform(onehot_pred1).toarray()
#print(test_pred1)
#print(onehot_pred1)

print(accuracy_score(test_pred1,test_label))
#print(clf.n_support_)
conf=confusion_matrix(test_pred1, test_label)
testlabel_count
#accuracy_per_label=[]
#accuracy_per_label=confusion_matrix(test_pred1, test_label)/testlabel_count
#accuracy_per_label.diagonal()

0.5652173913043478


array([5., 2., 1., 2., 3., 3., 2., 3., 3., 4., 8., 1., 2., 1., 2., 4.])

In [ ]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid()
clf.fit(X, y)
test_pred2=clf.predict(test_set)
print(accuracy_score(test_pred2,test_label))

onehot_pred2=test_pred2.reshape(-1,1)
onehot_pred2=ohc.fit_transform(onehot_pred2).toarray()
#print(onehot_pred2)
#confusion_matrix(test_pred2, test_label)

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
clf = RandomForestClassifier(80)
clf.fit(X, y)
test_pred3=clf.predict(test_set)
print(accuracy_score(test_pred3,test_label))
#print(confusion_matrix(test_pred3, test_label))
onehot_pred3=test_pred3.reshape(-1,1)
onehot_pred3=ohc.fit_transform(onehot_pred3).toarray()
confusion_matrix(test_pred3, test_label)
accuracy_per_label=[]
accuracy_per_label=confusion_matrix(test_pred3, test_label)/testlabel_count
accuracy_per_label.diagonal()
#print(onehot_pred3)

0.8137254901960784


array([0.90909091, 1.        , 0.        , 0.8       , 0.66666667,
       0.75      , 0.9       , 0.83333333, 0.66666667, 0.71428571,
       1.        , 0.        , 0.8       , 0.66666667, 1.        ,
       0.75      ])

In [2]:
#ensemble 1 : unanimous consensus
ensemble1_pred= onehot_pred1*onehot_pred2*onehot_pred3
#print(ensemble1_pred)
ensemble1_pred=np.argmax(ensemble1_pred,axis=1)
print(ensemble1_pred)

NameError: name 'onehot_pred1' is not defined

In [26]:
#ensemble 2 : majority consensus
ensemble2_pred= (onehot_pred1+onehot_pred2+onehot_pred3)/2
ensemble2_pred= ensemble2_pred.astype('int64') 
ensemble2_pred= ensemble2_pred.astype('float64') 
ensemble2_pred=np.argmax(ensemble2_pred,axis=1)
print(ensemble2_pred)
print(test_label)
print(accuracy_score(ensemble2_pred,test_label))


[ 7  9 10  9  9  7  9 10  9  6  7  1  6  6  0 12  6  5  7 12  3  6 12  1
 10 12  2 10  9  9 12  5  0  3 12  6  9  7  5  7  1 12  6  1  5 14  3 14
 13  8 14  8 14 13  0 12  5  8  6 12  1 14  1 14  9  6]
[12, 9, 10, 10, 9, 12, 10, 10, 9, 6, 7, 1, 6, 2, 0, 0, 6, 5, 7, 3, 4, 6, 12, 1, 12, 12, 12, 0, 9, 12, 6, 5, 0, 12, 12, 6, 7, 2, 5, 0, 3, 12, 6, 1, 5, 1, 10, 13, 13, 8, 14, 8, 14, 13, 0, 12, 5, 8, 6, 12, 1, 5, 8, 3, 12, 6]
0.6212121212121212
